In [191]:
import pandas as pd

# Preprocessing the raw data

In [192]:
crude_oil_df = pd.read_csv('data collection/Crude Oil WTI Futures Historical Data - daily data 10 years.csv')
gold_df = pd.read_csv('data collection/SPDR Gold Shares (GLD) - yahoo finance, 10 years.csv')
sp500_df = pd.read_csv('data collection/S&P 500 Historical Data - daily data 10 years.csv')
silver_df = pd.read_csv('data collection/Silver Futures Historical Data - daily data 10 years.csv')
corn_df = pd.read_csv('data collection/US Corn Futures Historical Data - daily data 10 years.csv')
dollar_rate_df = pd.read_csv('data collection/US Dollar Index Historical Data - daily data 10 years.csv')
#soybean_df = pd.read_csv('data collection/US Soybeans Futures Historical Data - daily data 10 years.csv')

cpi_df = pd.read_csv('data collection/Customer Price Index.csv')
cpi_df['DATE'] = pd.to_datetime(cpi_df['DATE'], format='%Y-%m-%d')
cpi_df = cpi_df[cpi_df['DATE'] >= '2013-01']
cpi_df.rename(columns={'CPIAUCSL':'CPI', 'DATE':'Date'}, inplace=True)
cpi_df['CPI'] = cpi_df['CPI'].shift(2)
ppi_df = pd.read_csv('data collection/Producer Price Index.csv')
ppi_df['DATE'] = pd.to_datetime(ppi_df['DATE'], format='%Y-%m-%d')
ppi_df.rename(columns={'PPIACO':'PPI', 'DATE':'Date'}, inplace=True)
ppi_df['PPI'] = ppi_df['PPI'].shift(2)
m1sl_df = pd.read_csv('data collection/M1SL - Money Supply, all data.csv')
m1sl_df['DATE'] = pd.to_datetime(m1sl_df['DATE'], format='%Y-%m-%d')
m1sl_df = m1sl_df[m1sl_df['DATE'] >= '2013-01']
# shifting by one
m1sl_df['M1SL'] = m1sl_df['M1SL'].shift(2)
m1sl_df.rename(columns={'DATE':'Date'}, inplace=True)
dfs_dict = {
    'crude_oil_df': crude_oil_df,
    'gold_df': gold_df,
    'sp500_df': sp500_df,
    'silver_df': silver_df,
    'dollar_rate_df': dollar_rate_df,
    'cpi_df': cpi_df,
    'ppi_df': ppi_df,
    'm1sl_df': m1sl_df
}

In [193]:
def replace_symbols(value):
    if isinstance(value, (int, float, complex)):
        return value
    if isinstance(value, str):
        if '%' in value:
            return float(value.replace('%', ''))
        elif 'K' in value:
            return float(value.replace('K', '')) * 1000
        elif 'M' in value:
            return float(value.replace('M', '')) * 1000000
        elif ',' in value:
            return float(value.replace(',', ''))
    return value

for df_name, df in dfs_dict.items():
    try:
        # Convert all columns except 'Date' using replace_symbols
        for col in df.columns:
            if col != 'Date':
                df[col] = df[col].apply(replace_symbols)
    
    except Exception as e:
        print(f"Error with {df_name}: {e}")
        continue

In [194]:
for df in dfs_dict.values():
    df['Date'] = pd.to_datetime(df['Date'])

# Rename columns to include dataframe name as prefix
for name, df in dfs_dict.items():
    prefix = name.replace('_df', '')  # Remove '_df' from dataframe name
    df.rename(columns=lambda x: f"{prefix}_{x}" if x != 'Date' else x, inplace=True)

# Start with the first dataframe
merged_df = list(dfs_dict.values())[0]
print(merged_df)
# Iteratively merge remaining dataframes
for df in list(dfs_dict.values())[1:]:
    merged_df = pd.merge(merged_df, df, on='Date', how='outer')
    
# Ensure the 'Date' column is in datetime format
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df = merged_df.sort_values(by='Date', ascending=True)


           Date  crude_oil_Price  crude_oil_Open  crude_oil_High  \
0    2024-07-29            75.56           77.28           77.69   
1    2024-07-28            77.27           77.47           77.48   
2    2024-07-26            77.16           78.35           78.60   
3    2024-07-25            78.28           77.50           78.47   
4    2024-07-24            77.59           77.45           78.19   
...         ...              ...             ...             ...   
3039 2013-01-08            93.15           93.32           93.80   
3040 2013-01-07            93.19           93.21           93.35   
3041 2013-01-04            93.09           92.86           93.21   
3042 2013-01-03            92.92           92.91           93.30   
3043 2013-01-02            93.12           91.78           93.87   

      crude_oil_Low  crude_oil_Vol.  crude_oil_Change %  
0             75.52             NaN               -2.21  
1             77.22             NaN                0.14  
2        

In [195]:
# Filter the DataFrame from 2014 onwards
merged_df = merged_df[merged_df['Date'].dt.year >= 2014]
cleaned_df = merged_df.dropna(axis=1, how='all')
# Display the filtered DataFrame
print(merged_df)

           Date  crude_oil_Price  crude_oil_Open  crude_oil_High  \
264  2014-01-01            98.70           98.61           98.76   
265  2014-01-02            95.44           98.50           98.97   
266  2014-01-03            93.96           95.47           95.74   
267  2014-01-06            93.43           94.18           94.59   
268  2014-01-07            93.67           93.60           94.22   
...         ...              ...             ...             ...   
3127 2024-08-26              NaN             NaN             NaN   
3128 2024-08-27              NaN             NaN             NaN   
3129 2024-08-28              NaN             NaN             NaN   
3130 2024-08-29              NaN             NaN             NaN   
3131 2024-08-30              NaN             NaN             NaN   

      crude_oil_Low  crude_oil_Vol.  crude_oil_Change %   gold_Open  \
264           98.54             NaN                0.28         NaN   
265           95.34        251250.0      

In [196]:
merged_df

,Date,crude_oil_Price,crude_oil_Open,crude_oil_High,crude_oil_Low,crude_oil_Vol.,crude_oil_Change %,gold_Open,gold_High,gold_Low,...,silver_Change %,dollar_rate_Price,dollar_rate_Open,dollar_rate_High,dollar_rate_Low,dollar_rate_Vol.,dollar_rate_Change %,cpi_CPI,ppi_PPI,m1sl_M1SL
264,2014-01-01,98.70,98.61,98.76,98.54,NaN,0.28,NaN,NaN,NaN,...,NaN,80.03,80.21,80.24,80.10,NaN,0.00,234.1,201.2,2638.8
265,2014-01-02,95.44,98.50,98.97,95.34,251250.0,-3.30,117.930000,118.730003,117.750000,...,3.91,80.63,80.12,80.71,80.08,NaN,0.75,NaN,NaN,NaN
266,2014-01-03,93.96,95.47,95.74,93.86,215520.0,-1.55,118.639999,119.620003,118.589996,...,0.41,80.79,80.57,80.89,80.49,NaN,0.20,NaN,NaN,NaN
267,2014-01-06,93.43,94.18,94.59,93.20,189380.0,-0.56,119.760002,120.389999,117.110001,...,-0.53,80.65,80.86,80.91,80.54,NaN,-0.17,NaN,NaN,NaN
268,2014-01-07,93.67,93.60,94.22,93.35,177180.0,0.26,118.459999,118.919998,118.129997,...,-1.57,80.83,80.69,80.95,80.60,NaN,0.22,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,2024-08-26,NaN,NaN,NaN,NaN,NaN,NaN,233.449997,233.500000,232.059998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3128,2024-08-27,NaN,NaN,NaN,NaN,NaN,NaN,231.690002,233.470001,231.570007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3129,2024-08-28,NaN,NaN,NaN,NaN,NaN,NaN,232.000000,232.020004,230.729996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3130,2024-08-29,NaN,NaN,NaN,NaN,NaN,NaN,231.949997,233.610001,231.919998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [197]:
# Filling NaNs of Sundays (there are Nans on Sundays in all columns except of soybean prices)
cleaned_df['is_holiday'] = 0
cleaned_df.loc[cleaned_df['gold_Close'].isna(), 'is_holiday'] = 1
cleaned_df = cleaned_df.fillna(method='ffill')

all_data_df = cleaned_df
all_data_df.to_csv('data collection/all_data.csv', index=False)
#only since 2014
all_data_df = all_data_df[all_data_df['Date'].dt.year >= 2014]

/var/folders/zp/tjm6hdl95rjgm9v39jqzvwb00000gq/T/ipykernel_43472/2576198648.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['is_holiday'] = 0
/var/folders/zp/tjm6hdl95rjgm9v39jqzvwb00000gq/T/ipykernel_43472/2576198648.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cleaned_df = cleaned_df.fillna(method='ffill')


In [198]:
all_data_df.to_csv('all_data.csv', index=False)

In [199]:
all_data_df = pd.read_csv('all_data.csv')
all_data_df

,Date,crude_oil_Price,crude_oil_Open,crude_oil_High,crude_oil_Low,crude_oil_Vol.,crude_oil_Change %,gold_Open,gold_High,gold_Low,...,silver_Change %,dollar_rate_Price,dollar_rate_Open,dollar_rate_High,dollar_rate_Low,dollar_rate_Change %,cpi_CPI,ppi_PPI,m1sl_M1SL,is_holiday
0,2014-01-01,98.70,98.61,98.76,98.54,NaN,0.28,NaN,NaN,NaN,...,NaN,80.03,80.21,80.24,80.10,0.00,234.100,201.200,2638.8,1
1,2014-01-02,95.44,98.50,98.97,95.34,251250.0,-3.30,117.930000,118.730003,117.750000,...,3.91,80.63,80.12,80.71,80.08,0.75,234.100,201.200,2638.8,0
2,2014-01-03,93.96,95.47,95.74,93.86,215520.0,-1.55,118.639999,119.620003,118.589996,...,0.41,80.79,80.57,80.89,80.49,0.20,234.100,201.200,2638.8,0
3,2014-01-06,93.43,94.18,94.59,93.20,189380.0,-0.56,119.760002,120.389999,117.110001,...,-0.53,80.65,80.86,80.91,80.54,-0.17,234.100,201.200,2638.8,0
4,2014-01-07,93.67,93.60,94.22,93.35,177180.0,0.26,118.459999,118.919998,118.129997,...,-1.57,80.83,80.69,80.95,80.60,0.22,234.100,201.200,2638.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2863,2024-08-26,75.56,77.28,77.69,75.52,365120.0,-2.21,233.449997,233.500000,232.059998,...,-1.93,104.66,104.37,104.67,104.14,0.27,313.207,256.772,17978.9,0
2864,2024-08-27,75.56,77.28,77.69,75.52,365120.0,-2.21,231.690002,233.470001,231.570007,...,-1.93,104.66,104.37,104.67,104.14,0.27,313.207,256.772,17978.9,0
2865,2024-08-28,75.56,77.28,77.69,75.52,365120.0,-2.21,232.000000,232.020004,230.729996,...,-1.93,104.66,104.37,104.67,104.14,0.27,313.207,256.772,17978.9,0
2866,2024-08-29,75.56,77.28,77.69,75.52,365120.0,-2.21,231.949997,233.610001,231.919998,...,-1.93,104.66,104.37,104.67,104.14,0.27,313.207,256.772,17978.9,0
